In [32]:
import cv2  #Open CV - Computer Vision
from keras.models import load_model 
import numpy as np
from collections import deque #Queue (Stack and Queue)
from PIL import Image

In [33]:
model1 = load_model('model/devanagari_model_refined.h5')
print(model1)

In [34]:
letter_count = {0: 'Check', 1: '01_ka', 2: '02_kha', 3: '03_ga', 4: '04_gha', 5: '05_kna',
               6: '06_cha', 7: '07_chha', 8: '08_ja', 9: '04_jha', 10: '10_yna',
               11: '11_taamatar', 12: '12_thaa', 13: '13_da', 14: '14_dhaa', 15: '15_adna',
               16: '16_tabala', 17: '17_tha', 18: '18_da', 19: '19_dha', 20: '20_na',
               21: '21_pa', 22: '22_pha', 23: '23_ba', 24: '24_bha', 25: '25_ma',
               26: '26_yaw', 27: '27_ra', 28: '28_la', 29: '29_waw', 30: '30_motosaw',
               31: '31_petchiryakha', 32: '32_patalosaw', 33: '33_ha', 34: '34_chhya', 35: '35_tra',
               36: '36_gya', 37: '37_Check'}

In [35]:
def keras_predict(model, image): #To predict data from webcam
    processed = keras_process_image(image)
    print("processed: "+str(processed.shape))
    pred_probab = model.predict(processed)[0]
    #Get class with max probability 
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class

#Gets image, resize image to a 32x32 pixel size. Then convert to np array and reshape/ 
def keras_process_image(img):
    image_x = 32
    image_y = 32
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    # -1 to get last value. X,y is height,width and 1 is number of channels
    img=  np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [42]:
cap = cv2.VideoCapture(0)  #Get webcam feed
Lower_blue = np.array([90, 50, 50])  #BGR format. Capture the blue colours
Upper_blue = np.array([130, 255, 255])  #When camera captures value between 110-130, then it's probably a blue, so that's what we want
pred_class=0 
pts = deque(maxlen=512) #queue
blackboard = np.zeros((480, 640, 3), dtype=np.uint8) #Actual drawings are placed in this blackboard
digit = np.zeros((200, 200, 3), dtype=np.uint8) 

while (True):  #While we are actually capturing anything
    ret, img = cap.read() #Get image from webcam
    img = cv2.flip(img, 1)  #Flip image coz it'll probably be mirrored
    imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #Convert to HSV
    mask = cv2.inRange(imgHSV, Lower_blue, Upper_blue) #Mask between lower bound and upper bound blue. If anything between these 2, we want
    blur = cv2.medianBlur(mask, 15) #Blue image to get rid of edges
    blur = cv2.GaussianBlur(blur, (5,5), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1] #ignore values beyond threshold
    cnts,_ = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[-2:] #diff features on image beyond threshold
    center = None

    if(cnts is None):
        print("None")
    elif(len(cnts) >= 1): #if contour is present (Blue colour is present)
        contour = max(cnts, key=cv2.contourArea) #Take max of contour and calc area
        if(cv2.contourArea(contour) > 250):
            ((x,y), radius) = cv2.minEnclosingCircle(contour) #enclose contour in circle to ensure where contour is 
            cv2.circle(img, (int(x), int(y)), int(radius), (0, 255, 255), 2)  #Creates circle around contour
            cv2.circle(img, center, 5, (0, 0, 255, -1))
            M = cv2.moments(contour)
            center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
            pts.appendleft(center) #Insers center in a queue. 
            for i in range(1, len(pts)):
                if(pts[i-1] is None or pts[i] is None):
                    continue
                #Draw line in blackboard of black color and length 10
                cv2.line(blackboard, pts[i-1], pts[i], (255, 255, 255), 10)
                #Draw line in image of red color and size 5
                cv2.line(img, pts[i-1], pts[i], (0, 0, 255), 5)
    elif(len(cnts) ==0): #Suppose we have removed blue colour on screen. No contours
        if(len(pts) != []):
            #Convert to gray colour, blur, get threshold and get contour
            blackboard_gray = cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY)
            blur1 = cv2.medianBlur(blackboard_gray, 15)
            blur1 = cv2.GaussianBlur(blur1, (5,5), 0)
            thresh1 = cv2.threshold(blur1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            blackboard_cnts, _ = cv2.findContours(thresh1.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[-2:]
            if(len(blackboard_cnts)) >=1:
                #Get max of contour
                cnt = max(blackboard_cnts, key=cv2.contourArea)
                #print area
                print(cv2.contourArea(cnt))
                #If area>2000, then it is noise. Might be blue color in background, etc
                if cv2.contourArea(cnt) > 2000:
                    x,y,w,h = cv2.boundingRect(cnt) #Bound contour into rectangle
                    #Convert blackboard gray image to digit variable
                    digit = blackboard_gray[y:y + h, x:x + w]
                    #newImage = process_letter(digit)
                    pred_probab, pred_class = keras_predict(model1, digit)
                    print(pred_class, pred_probab)

        pts = deque(maxlen=512)
        blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
    cv2.putText(img, "Conv network: "+str(letter_count[pred_class]), (10, 470),
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
    cv2.imshow("Contours", thresh)
    cv2.imshow("Frame", img)
    k = cv2.waitKey(1)
    if  k & 0xFF == 27 or ret==False:
        break

cv2.destroyAllWindows()
cap.release()
                

5214.0
processed: (1, 32, 32, 1)
1 0.47232386
7024.5
processed: (1, 32, 32, 1)
19 0.19123149
4079.0
processed: (1, 32, 32, 1)
21 0.32669625
4139.5
processed: (1, 32, 32, 1)
24 0.41786933
4185.5
processed: (1, 32, 32, 1)
25 0.2902307
270.5
3757.0
processed: (1, 32, 32, 1)
27 0.7268657
4999.0
processed: (1, 32, 32, 1)
28 0.2600375
